In [3]:
from datamodule import CT_Datamodule
from torchvision.transforms import functional as TF
import torch

In [4]:
d = CT_Datamodule("Dataset", num_workers=0)

In [5]:
d.prepare_data()
d.setup("fit")

In [7]:
def calculate_stats(loader: torch.utils.data.DataLoader):
    """
    Calculate the mean and standard deviation of the data in the loader

    Args:
        loader (torch.utils.data.DataLoader): The data loader

    Returns:
        Tuple[torch.Tensor]: The channel-wise mean and standard deviation of the data
    """
    psum = torch.tensor([0.0] * loader.dataset[0][2][0].shape[0])
    psum_sq = torch.tensor([0.0] * loader.dataset[0][2][0].shape[0])

    for _, _, inputs, _ in loader:
        inputs = inputs[0]
        psum += inputs.sum(axis=[0, 2, 3])
        psum_sq += (inputs**2).sum(axis=[0, 2, 3])

    count = (
        len(loader.dataset)
        * loader.dataset[0][2][0].shape[1]
        * loader.dataset[0][2][0].shape[2]
    )

    total_mean = psum / count
    total_var = (psum_sq / count) - (total_mean**2)
    total_std = torch.sqrt(total_var)
    return total_mean, total_std

In [8]:
calculate_stats(d.train_dataloader())

(tensor([0.1737, 0.1737, 0.1737]), tensor([0.2584, 0.2584, 0.2584]))

In [6]:
import torch

samples_reg = torch.zeros(4)
samples_tum = torch.zeros(4)

for i, j, _, _ in d.train:
    samples_reg += i
    samples_tum += j

for i, j, _, _ in d.val:
    samples_reg += i
    samples_tum += j

In [11]:
samples_tum

tensor([ 99., 362., 114.,  75.], dtype=torch.float64)

In [2]:
import torch

In [3]:
m= torch.hub.load("facebookresearch/dinov2", "dinov2_vitb14")

Using cache found in C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to C:\Users\Owais Makroo/.cache\torch\hub\checkpoints\dinov2_vitb14_pretrain.pth
100%|██████████| 330M/330M [00:10<00:00, 32.0MB/s] 


In [4]:
m

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [17]:
m

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [1]:
from model import Classifier

In [2]:
m = Classifier(1e-3, 2e-5, 1)

Using cache found in C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [5]:
from loss import SupervisedContrastiveLoss

In [6]:
l = SupervisedContrastiveLoss()

In [7]:
x = torch.randn(32, 16)
x = torch.concat([x.unsqueeze(1), x.unsqueeze(1)], dim=1)
x.shape

torch.Size([32, 2, 16])

In [12]:
l(x)

tensor(0.1657)

In [2]:
import json
import numpy as np

In [3]:
with open("results.json", "r") as f:
    data = json.load(f)

In [4]:
res = {key: [] for key in data["0"].keys()}

In [5]:
for i in data:
    for k in data[i]:
        res[k].append(data[i][k])

In [6]:
for k in res:
    res[k] = np.array(res[k])
    print(k, res[k].mean(), res[k].std())

val_loss 19.424339543788115 11.35818379459751
val_f1 0.5310939371585846 0.0570391465157692
val_auc 0.722384387254715 0.06477022132861987
val_acc 0.564117756485939 0.08067464243507903
val_rec 0.564117756485939 0.08067464243507903
val_acc_0 0.49425640925765035 0.22620277776124015
val_acc_1 0.7205801129341125 0.11028153409786116
val_acc_2 0.5709743678569794 0.10636846443750053
val_acc_3 0.3978461615741253 0.2689560374464342
val_f1_0 0.37585037648677827 0.12278642596907881
val_f1_1 0.7277752935886384 0.07073368233283246
val_f1_2 0.5706884235143661 0.0905883162652921
val_f1_3 0.3773333504796028 0.2124755841919272
val_auc_0 0.7277935326099396 0.10562361868561365
val_auc_1 0.7736896991729736 0.04758807745181345
val_auc_2 0.7712440669536591 0.07989227453259277
val_auc_3 0.6168102502822876 0.2224773177507702
val_rec_0 0.49425640925765035 0.22620277776124015
val_rec_1 0.7205801129341125 0.11028153409786116
val_rec_2 0.5709743678569794 0.10636846443750053
val_rec_3 0.3978461615741253 0.2689560374